## Environment request

In [1]:
#!pip install tensorflow-gpu==2.0.0

In [2]:
#!pip uninstall -y tensorflow

In [3]:
# !pip install pydot-ng 
# !pip install graphviz 
# !pip install pydot==1.2.3

## Select GPU for training 

In [4]:
import tensorflow as tf

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.0.0
Num GPUs Available:  2


In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

2 Physical GPUs, 1 Logical GPU


## Import request package

In [6]:
import os
import cv2
import glob
import shutil
import random

import numpy as np
import matplotlib.pyplot as plt

from model import *
from data import *
from tqdm import tqdm
from datetime import datetime



import tensorflow as tf
tf.__version__

Using TensorFlow backend.


'2.0.0'

## K-fold-learning hyperparameter

In [7]:
k = 6
#K-fold default

train_image_folder = '/home/jingping/aiforge/tooth_xray_segmention_for_test/unet_base/dataset/train_tooth/train/image'
train_label_folder = '/home/jingping/aiforge/tooth_xray_segmention_for_test/unet_base/dataset/train_tooth/train/label'
k_fold_data_save_path = '/home/jingping/aiforge/tooth_xray_segmention_for_test/unet_base/dataset/train_tooth'
k_fold_folder_name = 'k_fold_learning'
#load data and split K group

model_save_path = '/home/jingping/aiforge/tooth_xray_segmention_for_test/unet_base/model/model_weight/tooth_background_bi_k-fold/'
model_save_name = 'tooth_background_bi_k-fold'
model_save_tpye = '.hdf5'
# model will be save in 'model_save_path+model_save_name_{K}+model_save_tpye


## K-fold folder build

In [8]:
# if show create finish!! can keep going next step
k_fold_folder_path = os.path.join(k_fold_data_save_path, k_fold_folder_name)

if not os.path.isdir(k_fold_folder_path):
    try:
        os.makedirs(k_fold_folder_path)
        os.makedirs(model_save_path)
        for i in range(k):
            os.makedirs(os.path.join(k_fold_folder_path,'group{}'.format(i+1)))
            os.makedirs(os.path.join(k_fold_folder_path,'group{}/image'.format(i+1)))
            os.makedirs(os.path.join(k_fold_folder_path,'group{}/label'.format(i+1)))
            os.makedirs(os.path.join(k_fold_folder_path,'group{}/aug'.format(i+1)))
            os.makedirs(os.path.join(k_fold_folder_path,'group{}/skip_data'.format(i+1)))
            os.makedirs(os.path.join(k_fold_folder_path,'group{}/skip_data/image'.format(i+1)))
            os.makedirs(os.path.join(k_fold_folder_path,'group{}/skip_data/label'.format(i+1)))
            os.makedirs(os.path.join(k_fold_folder_path,'group{}/skip_data/prediction'.format(i+1)))
            
        print('{} create finish!!'.format(k_fold_folder_name))
    except:
        print('{} build error'.format(k_fold_folder_name))
        os.rmdir(k_fold_folder_path)
        os.rmdir(model_save_path)
else:
    print('{} already exist'.format(k_fold_folder_name))

k_fold_learning create finish!!


## Assign data in k group

In [9]:
traing_image_list = glob.glob(train_image_folder+'/*.PNG')
#traing_image_list.sort()
random.shuffle (traing_image_list)

total_data_len = len(traing_image_list)
batch_num = total_data_len/k

for index,img_path in enumerate(tqdm(traing_image_list, total = total_data_len)):
    #data in other group 
    group_list = [i+1 for i in range(k)]
    group_list.pop(int(index/batch_num))
    for group in group_list:
        img_dst = os.path.join(k_fold_folder_path,'group{}/image'.format(group))
        shutil.copy2(img_path,img_dst)
        label_path = img_path.replace(train_image_folder,train_label_folder)
        label_dst = img_dst.replace('image','label')
        shutil.copy2(label_path,label_dst)
    
    
    #put data in skip_data path 
    skip_img_dst = os.path.join(k_fold_folder_path,'group{}/skip_data/image'.format(int(index/batch_num)+1))
    shutil.copy2(img_path,skip_img_dst)
    skip_label_path = img_path.replace(train_image_folder,train_label_folder)
    skip_label_dst = skip_img_dst.replace('image','label')
    shutil.copy2(skip_label_path,skip_label_dst)
print('Data in {} groups'.format(k))

100%|██████████| 68/68 [00:00<00:00, 332.53it/s]

Data in 6 groups


## Data augmentation in k group

In [10]:
def Data_augmentation(train_parent_folder,aug_path):
    data_gen_args = dict(rotation_range=0.2,
                        width_shift_range=0.05,
                        height_shift_range=0.05,
                        shear_range=0.05,
                        zoom_range=0.05,
                        horizontal_flip=True,
                        fill_mode='nearest')
    myGenerator = trainGenerator(20,train_parent_folder,'image','label',data_gen_args,save_to_dir = aug_path)
    
    num_batch = 3
    for i,batch in enumerate(myGenerator):
        if(i >= num_batch):
            break
    #image_arr,mask_arr = geneTrainNpy(aug_path,aug_path)

In [11]:
k_fold_folder_path = os.path.join(k_fold_data_save_path, k_fold_folder_name)
group_list = os.listdir(k_fold_folder_path)
group_list.sort()

for group_name in tqdm(group_list, total = len(group_list)):
    print('This is {}'.format(group_name))
    group_path = os.path.join(k_fold_folder_path, group_name)
    aug_path = os.path.join(group_path, 'aug')
    Data_augmentation(group_path, aug_path)

  0%|          | 0/6 [00:00<?, ?it/s]

This is group1
Found 56 images belonging to 1 classes.
Found 56 images belonging to 1 classes.


 17%|█▋        | 1/6 [00:02<00:11,  2.24s/it]

This is group2
Found 57 images belonging to 1 classes.
Found 57 images belonging to 1 classes.


 33%|███▎      | 2/6 [00:04<00:08,  2.24s/it]

...

 67%|██████▋   | 4/6 [00:08<00:04,  2.23s/it]

This is group5
Found 57 images belonging to 1 classes.
Found 57 images belonging to 1 classes.


 83%|████████▎ | 5/6 [00:11<00:02,  2.22s/it]

This is group6
Found 57 images belonging to 1 classes.
Found 57 images belonging to 1 classes.


100%|██████████| 6/6 [00:13<00:00,  2.26s/it]


# Train Unet with training data 

In [12]:
def train_in_traindata_unet(train_parent_folder,model_save_path, model_checkpoint):
    data_gen_args = dict(rotation_range=0.2,
                        width_shift_range=0.05,
                        height_shift_range=0.05,
                        shear_range=0.05,
                        zoom_range=0.05,
                        horizontal_flip=True,
                        fill_mode='nearest')
    myGene = trainGenerator(3,train_parent_folder,'image','label',data_gen_args,save_to_dir = None)
    model = unet()
    model.fit_generator(myGene,steps_per_epoch=2000,epochs=5,callbacks=[model_checkpoint])
    return model

In [13]:
# model.summary()

# Train Unet with augmentation data

In [14]:
def train_in_augdata_unet(model, aug_path, model_save_path, model_checkpoint):    
    imgs_train,imgs_mask_train = geneTrainNpy(aug_path, aug_path)
    model.fit(imgs_train, imgs_mask_train, batch_size=2, nb_epoch=5, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])
    model.save(model_save_path)

In [15]:
k_fold_folder_path = os.path.join(k_fold_data_save_path, k_fold_folder_name)
group_list = os.listdir(k_fold_folder_path)
group_list.sort()

for group_name in tqdm(group_list, total = len(group_list)):
    print('Model start training in {}'.format(group_name))
    group_path = os.path.join(k_fold_folder_path, group_name)
    aug_path = os.path.join(group_path, 'aug')
    model_save_total_path = model_save_path+model_save_name+'_{}'.format(group_name)+model_save_tpye
    model_checkpoint = ModelCheckpoint(model_save_total_path, monitor='loss',verbose=1, save_best_only=True)
    model = train_in_traindata_unet(group_path, model_save_total_path, model_checkpoint)
    train_in_augdata_unet(model, aug_path, model_save_total_path, model_checkpoint)

  0%|          | 0/6 [00:00<?, ?it/s]

Model start training in group1


/home/jingping/aiforge/tooth_xray_segmention_for_test/unet_base/model/unet/model.py:56: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


Epoch 1/5
Found 56 images belonging to 1 classes.
Found 56 images belonging to 1 classes.
2000/2000 [==============================] - 165s 83ms/step - loss: 0.4473 - accuracy: 0.8427

Epoch 00001: loss improved from inf to 0.44737, saving model to /home/jingping/aiforge/tooth_xray_segmention_for_test/unet_base/model/model_weight/tooth_background_bi_k-fold/tooth_background_bi_k-fold_group1.hdf5
Epoch 2/5
2000/2000 [==============================] - 157s 78ms/step - loss: 0.2967 - accuracy: 0.9537

Epoch 00002: loss improved from 0.44737 to 0.29680, saving model to /home/jingping/aiforge/tooth_xray_segmention_for_test/unet_base/model/model_weight/tooth_background_bi_k-fold/tooth_background_bi_k-fold_group1.hdf5
Epoch 3/5
2000/2000 [==============================] - 157s 78ms/step - loss: 0.2456 - accuracy: 0.9694

Epoch 00003: loss improved from 0.29680 to 0.24567, saving model to /home/jingping/aiforge/tooth_xray_segmention_for_test/unet_base/model/model_weight/tooth_background_bi_k-fo

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


...

 67%|██████▋   | 4/6 [54:04<27:00, 810.33s/it]

Model start training in group5
Epoch 1/5
Found 57 images belonging to 1 classes.
Found 57 images belonging to 1 classes.
2000/2000 [==============================] - 160s 80ms/step - loss: 0.2150 - accuracy: 0.9035

Epoch 00001: loss improved from inf to 0.21496, saving model to /home/jingping/aiforge/tooth_xray_segmention_for_test/unet_base/model/model_weight/tooth_background_bi_k-fold/tooth_background_bi_k-fold_group5.hdf5
Epoch 2/5
2000/2000 [==============================] - 158s 79ms/step - loss: 0.0736 - accuracy: 0.9685

Epoch 00002: loss improved from 0.21496 to 0.07362, saving model to /home/jingping/aiforge/tooth_xray_segmention_for_test/unet_base/model/model_weight/tooth_background_bi_k-fold/tooth_background_bi_k-fold_group5.hdf5
Epoch 3/5
2000/2000 [==============================] - 159s 79ms/step - loss: 0.0486 - accuracy: 0.9793

Epoch 00003: loss improved from 0.07362 to 0.04857, saving model to /home/jingping/aiforge/tooth_xray_segmention_for_test/unet_base/model/model_

 83%|████████▎ | 5/6 [1:07:37<13:31, 811.15s/it]

Model start training in group6
Epoch 1/5
Found 57 images belonging to 1 classes.
Found 57 images belonging to 1 classes.
2000/2000 [==============================] - 160s 80ms/step - loss: 0.6904 - accuracy: 0.5561

Epoch 00001: loss improved from inf to 0.69037, saving model to /home/jingping/aiforge/tooth_xray_segmention_for_test/unet_base/model/model_weight/tooth_background_bi_k-fold/tooth_background_bi_k-fold_group6.hdf5
Epoch 2/5
2000/2000 [==============================] - 159s 79ms/step - loss: 0.6877 - accuracy: 0.5561

Epoch 00002: loss improved from 0.69037 to 0.68766, saving model to /home/jingping/aiforge/tooth_xray_segmention_for_test/unet_base/model/model_weight/tooth_background_bi_k-fold/tooth_background_bi_k-fold_group6.hdf5
Epoch 3/5
2000/2000 [==============================] - 159s 79ms/step - loss: 0.6870 - accuracy: 0.5562

Epoch 00003: loss improved from 0.68766 to 0.68698, saving model to /home/jingping/aiforge/tooth_xray_segmention_for_test/unet_base/model/model_

100%|██████████| 6/6 [1:21:12<00:00, 812.00s/it]


### test your model and save predicted results

In [16]:
from model import *
from data import *
from datetime import datetime
from tqdm import tqdm

import os
import tensorflow as tf
tf.__version__

'2.0.0'

In [17]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

2 Physical GPUs, 1 Logical GPU


In [18]:
def ensemble_bin(img, model_list, flip_ = False):
    ensemble_final_result = np.zeros((img.shape[0], img.shape[1],1), dtype = 'float32')
    for index, model in enumerate(model_list):
        pr_img = model.predict(img)
        sqpr_img = np.squeeze(pr_img)
        ensemble_img = sqpr_img.copy()
        if flip_:
            imgp = np.squeeze(img)
            h_flip = cv2.flip(imgp, 1)
            h_flip = np.expand_dims(h_flip, axis=(0,3))
            pre_h_flip = model.predict(h_flip)
            pre_h_flip = np.squeeze(pre_h_flip)
            h_flip = cv2.flip(pre_h_flip, 1)

            v_flip = cv2.flip(imgp, 0)
            v_flip = np.expand_dims(v_flip, axis=(0,3))
            pre_v_flip = model.predict(v_flip)
            pre_v_flip = np.squeeze(pre_v_flip)
            v_flip = cv2.flip(pre_v_flip, 0)

            hv_flip = cv2.flip(imgp, -1)
            hv_flip = np.expand_dims(hv_flip, axis=(0,3))
            pre_hv_flip = model.predict(hv_flip)
            pre_hv_flip = np.squeeze(pre_hv_flip)
            hv_flip = cv2.flip(pre_hv_flip, -1)
            ensemble_img = (sqpr_img+h_flip+v_flip+hv_flip)/4
        ensemble_final_result = ensemble_final_result+ensemble_img
    ensemble_final_result = ensemble_final_result/(index+1)
    return np.expand_dims(np.where(ensemble_final_result > 0.5, 255, 0), axis=(0,3))

In [29]:
import os 
import shutil

import skimage.io as io
import skimage.transform as trans

from tqdm import tqdm

def test_k_group_in_skip_data(test_folder, result_folder, model_save_path):
    model = unet()
    model.load_weights(model_save_path)
    model_list = [model]
    model_fit_img_size = (256,256)
    test_img_path_list = glob.glob(test_folder+'/*.PNG')
    for img_path in tqdm(test_img_path_list, total = len(test_img_path_list)):
        img = io.imread(img_path,as_gray = True)
        targe_size = img.shape
        img = trans.resize(img,model_fit_img_size)
        img = np.expand_dims(img, axis=(0,3))
        img_esemble = ensemble_bin(img,model_list,flip_ = True)
        img_esemble = np.squeeze(img_esemble)
        cv2.imwrite(result_folder+'/{}'.format(img_path.split('/')[-1]),img_esemble)

In [30]:
k_fold_folder_path = os.path.join(k_fold_data_save_path, k_fold_folder_name)
group_list = os.listdir(k_fold_folder_path)
group_list.sort()

for group_name in tqdm(group_list, total = len(group_list)):
    test_folder = os.path.join(k_fold_folder_path, group_name, 'skip_data', 'image')
    result_folder = os.path.join(k_fold_folder_path, group_name, 'skip_data', 'prediction')
    model_save_total_path = model_save_path+model_save_name+'_{}'.format(group_name)+model_save_tpye
    test_k_group_in_skip_data(test_folder, result_folder, model_save_total_path)

100%|██████████| 6/6 [00:08<00:00,  1.34s/it]


In [31]:
# !conda install -y graphviz

In [40]:
#show testing result
import os
import cv2
import skimage.io as io
import skimage.transform as trans
import matplotlib.pyplot as plt


def show_k_fold_result(test_img_path_list):
    for img_path in tqdm(test_img_path_list, total = len(test_img_path_list)):
        img = cv2.imread(img_path,0)
        predict_img = cv2.imread(img_path.replace('image','prediction'),0)
        print(img.shape[::-1])
        predict_img = cv2.resize(predict_img, img.shape[::-1][1:])
        human_label = cv2.imread(img_path.replace('image','label'),0)
        plt.figure(figsize = (20,20))
        plt.subplot(131)
        plt.title('original_test_img')
        plt.imshow(img, cmap='gray')
        plt.subplot(132)
        plt.title('predict_test_img')
        plt.imshow(predict_img, cmap='gray')
        plt.subplot(133)
        plt.title('human_label')
        plt.imshow(human_label, cmap='gray')
        plt.show()

In [39]:
k_fold_folder_path = os.path.join(k_fold_data_save_path, k_fold_folder_name)
group_list = os.listdir(k_fold_folder_path)
group_list.sort()

for group_name in tqdm(group_list, total = len(group_list)):
    test_folder = os.path.join(k_fold_folder_path,group_name, 'skip_data', 'image')
    test_img_path_list = glob.glob(test_folder+'/*.PNG')
    show_k_fold_result(test_img_path_list)

  0%|          | 0/6 [00:00<?, ?it/s]

(615,)


TypeError: function takes exactly 2 arguments (1 given)

In [24]:
k_fold_folder_path = os.path.join(k_fold_data_save_path, k_fold_folder_name)
group_list = os.listdir(k_fold_folder_path)
group_list.sort()

for group_name in tqdm(group_list, total = len(group_list)):
    test_folder = os.path.join(k_fold_folder_path, group_name, 'skip_data', 'image')
    result_folder = os.path.join(k_fold_folder_path, group_name, 'skip_data', 'prediction')
    model_save_total_path = model_save_path+model_save_name+'_{}'.format(group_name)+model_save_tpye
    test_k_group_in_skip_data(test_folder, result_folder, model_save_total_path)

100%|██████████| 6/6 [00:05<00:00,  1.03it/s]


In [25]:
test_folder = '/home/jingping/aiforge/tooth_xray_segmention_for_test/unet_base/dataset/buccal_furcation_train/k_fold_learning/group5/image'
result_folder = '/home/jingping/aiforge/tooth_xray_segmention_for_test/unet_base/dataset/buccal_furcation_train/k_fold_learning/group5/train_pre'
model_save_total_path = '/home/jingping/aiforge/tooth_xray_segmention_for_test/unet_base/model/model_weight/k-fold/buccal_furcation_bi_unet_k-fold-learning_group5.hdf5'

test_k_group_in_skip_data(test_folder, result_folder, model_save_total_path)

100%|██████████| 101/101 [00:01<00:00, 54.26it/s]


In [26]:
import sys 
sys.path

['/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '',
 '/home/jingping/.local/lib/python3.6/site-packages',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/home/jingping/.ipython']